In [3]:
!pip install bs4==4.10.0 -y
!pip install nbformat==4.2.0
!mamba install html5lib==1.1 -y


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -y


'mamba' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)
url = "https://www.cbr.ru/eng/hd_base/mrrf/mrrf_7d/?UniDbQuery.Posted=True&UniDbQuery.From=01.2021&UniDbQuery.To=11.2024.html"
data  = requests.get(url).text
soup = BeautifulSoup(data, 'html.parser')

# Instead of directly accessing tbody, find the table first and then iterate through its rows
table = soup.find('table', {'class': 'data'}) 
# Check if the table is found
if table:
    reserve_data = pd.DataFrame(columns=["Date", "Volume"])
    for row in table.find_all('tr')[1:]:  # Skip the header row
        col = row.find_all("td")
        # Check if the expected number of columns is present
        if len(col) >= 2:
            Date = col[0].text
            Volume = col[1].text
            reserve_data = pd.concat([reserve_data, pd.DataFrame({"Date":[Date], "Volume":[Volume]})], ignore_index=True)
        else:
            print("Skipping row with insufficient columns:", row) # Print debug information
else:
    print("Table not found on the page.")

In [7]:
reserve_data.to_csv("reserve.csv", index=False)

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)
url = "https://www.cbr.ru/eng/hd_base/mb_nd/mb_nd_month/?UniDbQuery.Posted=True&UniDbQuery.From=01.2021&UniDbQuery.To=11.2024.html"
data  = requests.get(url).text
soup = BeautifulSoup(data, 'html.parser')

# Instead of directly accessing tbody, find the table first and then iterate through its rows
table = soup.find('table', {'class': 'data'}) 
# Check if the table is found
if table:
    monetary_data = pd.DataFrame(columns=["Date", "Monetary_Base", "Currency", "Reserve"])
    # Extract header names from the table for 'Currency' and 'Reserve'
    header_row = table.find_all('tr')[0]  # Get the header row
    header_cols = header_row.find_all('th')  # Get all th elements in the header row
    #currency_header = header_cols[2].text.strip()  # Extract Currency header text # currency_header is set to 'Date', this is incorrect.
    #reserve_header = header_cols[3].text.strip()  # Extract Reserve header text # reserve_header is set to 'Monetary Base', this is incorrect
    

    for row in table.find_all('tr')[2:]:  # Skip the header row and the next row
        col = row.find_all("td")
        # Check if the expected number of columns is present
        if len(col) >= 4: 
            Date = col[0].text
            Monetary_Base = col[1].text
            Currency = col[2].text 
            Reserve = col[3].text
            # Use the correct column names directly when creating the temporary DataFrame:
            monetary_data = pd.concat([monetary_data, pd.DataFrame({"Date":[Date], "Monetary_Base":[Monetary_Base], 
                                                                    "Currency":[Currency], "Reserve":[Reserve]})], ignore_index=True) 
        else:
            print

In [9]:
monetary_data.to_csv("monetary.csv", index=False)

In [10]:
url = "https://www.cbr.ru/eng/hd_base/deposit_base/?UniDbQuery.Posted=True&UniDbQuery.From=01.01.2021&UniDbQuery.To=20.11.2024.html"
data  = requests.get(url).text
soup = BeautifulSoup(data, 'html.parser')

# Instead of directly accessing tbody, find the table first and then iterate through its rows
table = soup.find('table', {'class': 'data'})  
# Check if the table is found
if table:
    interest_data = pd.DataFrame(columns=["Date", "Overnight"])
    for row in table.find_all('tr')[1:]:  # Skip the header row
        col = row.find_all("td")
        # Check if the expected number of columns is present
        if len(col) >= 2:
            Date = col[0].text
            Overnight = col[1].text
            interest_data = pd.concat([interest_data, pd.DataFrame({"Date":[Date], "Overnight":[Overnight]})], ignore_index=True)
        else:
            print("Skipping row with insufficient columns:", row) # Print debug information
else:
    print("Table not found on the page.")
           

In [11]:
interest_data.to_csv("interest.csv", index=False)

In [12]:
import requests
import pandas as pd

url = "https://www.investing.com/economic-calendar/russian-cpi-1180"

# Fetch the page content
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    # Add Referer header
    "Referer": "https://www.investing.com/" 
}  # Add a User-Agent and Referer to mimic a browser request

response = requests.get(url, headers=headers)

# Instead of raising an exception, print the status code for debugging
print(response.status_code)  

# Only proceed if the request was successful
if response.status_code == 200:
    try:
        cpi = pd.read_html(response.content, attrs={"id": "eventHistoryTable1180"})[0]
        # Rename columns for clarity (optional)
        cpi.columns = ["release_date", "time", "actual", "forecast", "previous"]
        print(cpi)
    except IndexError:
        print("Table not found on the page.")
else:
    print(f"Request failed with status code: {response.status_code}")

403
Request failed with status code: 403
